In [1]:
!pip install torch transformers datasets

  Using cached datasets-2.20.0-py3-none-any.whl.metadata (19 kB)
  Using cached pyarrow_hotfix-0.6-py3-none-any.whl.metadata (3.6 kB)
  Using cached dill-0.3.8-py3-none-any.whl.metadata (10 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached xxhash-3.4.1-cp312-cp312-win_amd64.whl.metadata (12 kB)
  Using cached multiprocess-0.70.16-py312-none-any.whl.metadata (7.2 kB)
  Using cached aiohttp-3.10.0-cp312-cp312-win_amd64.whl.metadata (7.8 kB)
  Using cached aiohappyeyeballs-2.3.4-py3-none-any.whl.metadata (5.6 kB)
  Using cached aiosignal-1.3.1-py3-none-any.whl.metadata (4.0 kB)
  Using cached frozenlist-1.4.1-cp312-cp312-win_amd64.whl.metadata (12 kB)
  Using cached multidict-6.0.5-cp312-cp312-win_amd64.whl.metadata (4.3 kB)
  Using cached yarl-1.9.4-cp312-cp312-win_amd64.whl.metadata (32 kB)
Using cached datasets-2.20.0-py3-none-any.whl (547 kB)
Using cached dill-0.3.8-py3-none-any.whl (116 kB)
Using cached aiohttp-3.10.0-cp312-cp312-win_amd64.whl (373 

In [2]:
from datasets import load_dataset

# Load the SCIQ dataset
dataset = load_dataset('sciq')


c:\Users\Ashwin\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating test split: 100%|██████████| 1000/1000 [00:00<00:00, 297595.00 examples/s]


In [9]:
from datasets import DatasetDict

# Define the percentage for training
train_percentage = 0.10

# Split the dataset
# Assuming `dataset['train']` is the training split
train_dataset = dataset['train'].train_test_split(test_size=1 - train_percentage)

# Now you have two datasets: 'train' and 'test'
train_data = train_dataset['train']
test_data = train_dataset['test']
dataset = train_data


In [10]:
print(f"Training data size: {len(train_data)}")
print(f"Test data size: {len(test_data)}")


Training data size: 1167
Test data size: 10512


In [11]:
from transformers import GPT2Tokenizer

# Load the GPT2 tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

# Add a padding token if not already set
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    tokenizer.pad_token_id = tokenizer.get_vocab()['[PAD]']

def tokenize_function(examples):
    return tokenizer(
        examples['question'],
        padding='max_length',
        truncation=True,
        max_length=128  # Set the desired maximum length
    )

# Tokenize the dataset with padding and truncation
tokenized_dataset = dataset.map(tokenize_function, batched=True)


Map: 100%|██████████| 1167/1167 [00:00<00:00, 2205.05 examples/s]


In [12]:
from transformers import GPT2LMHeadModel

model = GPT2LMHeadModel.from_pretrained('gpt2')

In [19]:
!pip install --upgrade accelerate

In [17]:
!python --version

Python 3.12.4


In [1]:
from transformers import Trainer, TrainingArguments, DataCollatorForLanguageModeling

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    evaluation_strategy="epoch",
    save_strategy="epoch",
    fp16=True,  # Enable mixed precision training if your GPU supports it
)

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,
)


c:\Users\Ashwin\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\Ashwin\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


NameError: name 'tokenizer' is not defined

In [ ]:
trainer = Trainer(
    model=model.to('cuda'),
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['validation'],
)

trainer.train()


In [ ]:
model.save_pretrained('./trained_model')
tokenizer.save_pretrained('./trained_model')


In [ ]:
trainer.evaluate()

In [ ]:
from transformers import pipeline

model = GPT2LMHeadModel.from_pretrained('./trained_model').to('cuda')
tokenizer = GPT2Tokenizer.from_pretrained('./trained_model')

qa_pipeline = pipeline('text-generation', model=model, tokenizer=tokenizer, device=0)

def answer_question(question):
    result = qa_pipeline(question, max_length=50, num_return_sequences=1)
    return result[0]['generated_text']

# Example usage
question = "What is the boiling point of water?"
print(answer_question(question))
